In [1]:
import pandas as pd
from pyspark import SparkContext
import time
import numpy as np
# from sklearn.cluster import KMeans
# import matplotlib.pyplot as plt
# import seaborn as sns
startTime = time.time()

In [2]:
sc = SparkContext("local[*]", "UserUserRating")

In [84]:
trainData = sc.textFile("/Users/apple/Desktop/DataMining/project/user_business_rating_user_sim.csv")
trainDataFirst = trainData.first()
trainData = trainData.filter(lambda x: x != trainDataFirst)
trainData = trainData.map(lambda l: l.split(','))
trainData = trainData.map(lambda x : (x[1],x[2],x[3])).collect()

In [4]:
#trainData = trainData[:40000]

In [85]:
users = {}
businesses = {}
finalInput = []
ratings = []

In [86]:
for i in trainData:
    if users.get(i[0]) is None:
        users[i[0]] = len(users) + 1
    if businesses.get(i[1]) is None:
        businesses[i[1]] = len(businesses) + 1
    ratings.append(float(i[2].strip()))
    finalInput.append([users[i[0]],businesses[i[1]],float(i[2].strip())])

In [87]:
invUsers = {str(v): k for k, v in users.items()}
invBusinesses = {str(v): k for k, v in businesses.items()}

trainAvgRating = [float(sum(col))/len(col) for col in zip(*finalInput)][2]
minVal = min(ratings)
maxVal = max(ratings)


In [88]:
finalTrainInput = sc.parallelize(finalInput)

userAverage = finalTrainInput.map(lambda l: (l[0], l[2])).mapValues(lambda l: (l,1)).reduceByKey(lambda x,y: (x[0] + y[0], x[1] + y[1]))
userAverage = userAverage.mapValues(lambda l: 1.0*l[0]/l[1]).collectAsMap()

businesAverage = finalTrainInput.map(lambda l: (l[1], l[2])).mapValues(lambda l: (l,1)).reduceByKey(lambda x,y: (x[0] + y[0], x[1] + y[1]))
businesAverage = businesAverage.mapValues(lambda l: 1.0*l[0]/l[1]).collectAsMap()

In [89]:
trainRdd = finalTrainInput
trainRddUsers = trainRdd.map(lambda x : ((str(x[0]), str(x[1])), (float(x[2])-minVal)/(maxVal-minVal)))
trainingRddUsers = trainRddUsers.map(lambda x : (x[0][0], (x[0][1], x[1])))
businessesOnlyUserCF = trainingRddUsers.map(lambda x : (x[1][0], x[0]))
businessesOnlyUserCF = businessesOnlyUserCF.groupByKey()
businessesOnlyUserCF = businessesOnlyUserCF.mapValues(list)
businessesOnlyUserCF = businessesOnlyUserCF.collectAsMap()
userAndBusinessesUserCF = trainingRddUsers.map(lambda x : ((x[0], x[1][0]), x[1][1]))
userAndBusinessesUserCF = userAndBusinessesUserCF.collectAsMap()

usersOnlyUserCF = trainingRddUsers.groupByKey()
usersOnlyUserCF = usersOnlyUserCF.mapValues(list)
usersOnlyUserCF = usersOnlyUserCF.collectAsMap()

In [90]:
def calcPearsonUsers(corratedUsersBusinessess):
    if len(corratedUsersBusinessess) == 0:
        return -100
    if len(corratedUsersBusinessess) == 1:
        return -100
    tempUB = [x[1][0] for x in corratedUsersBusinessess]
    tempUBAvg = sum(tempUB)/len(tempUB)
    pearsonNumerator = 0
    pearsonDenom1 = 0
    pearsonDenom2 = 0
    allUB = [x[1][1] for x in corratedUsersBusinessess]
    for (i, j) in zip(tempUB, allUB):
        pearsonNumerator += (i-tempUBAvg)*(j-tempUBAvg)
        pearsonDenom1 += (i-tempUBAvg)**2
        pearsonDenom2 += (j-tempUBAvg)**2
    pearsonDenom = (pearsonDenom1**0.5) * (pearsonDenom2**0.5)
    if pearsonDenom == 0 or pearsonNumerator == 0:
        return -100
    else:
        return pearsonNumerator/pearsonDenom

In [91]:
user_ids = usersOnlyUserCF.keys()

In [173]:
user_vs_business ={}
import pandas as pd
for k , v in usersOnlyUserCF.items():
    user_vs_business [k] =  len(v)
data = {"user" : user_vs_business.keys() , "count_business" : user_vs_business.values()}
df =  pd.DataFrame.from_dict(data)
df_new = df[ (df.count_business <14 ) & ( df.count_business >7 )]


In [174]:
user_ids =df_new.user.unique()

In [175]:
len(user_ids)

16220

In [176]:
pearson_between_users= []

In [ ]:
for i,user1 in enumerate(user_ids[:-1]):
    for j , user2 in enumerate(user_ids[i+1:]):
        eachUserInfo = usersOnlyUserCF[user1]
        tempKnnUsersInfo = usersOnlyUserCF[user2]
        corratedUsersBusinessess = []
        tempKnnUsersInfo.sort()
        eachUserInfo.sort()
        l= 0
        k = 0
        while l<len(tempKnnUsersInfo) and k< len(eachUserInfo):
            if tempKnnUsersInfo[l][0] == eachUserInfo[k][0]:
                corratedUsersBusinessess.append((tempKnnUsersInfo[l][0], (tempKnnUsersInfo[l][1], eachUserInfo[k][1])))
                l = l+1
                k = k+1
            elif tempKnnUsersInfo[l][0] < eachUserInfo[k][0]:
                l = l+1
            else:
                k = k+1
       
        pearsonRating = calcPearsonUsers(corratedUsersBusinessess)
        if pearsonRating != -100:
            pearson_between_users.append((user1,user2,pearsonRating))

In [140]:
pearson_between_users

[('278', '44709', 0.7071067811865475),
 ('12014', '44170', -0.7071067811865475),
 ('12016', '19859', -0.4472135954999579),
 ('46792', '10102', 0.9999999999999998),
 ('46792', '15685', 0.7071067811865475),
 ('20254', '48540', 0.6708203932499367),
 ('29927', '34102', -0.4472135954999579),
 ('12838', '31386', 0.7071067811865475),
 ('31163', '34125', -0.5999999999999999),
 ('30331', '16638', -0.447213595499958),
 ('35212', '9105', -0.7071067811865475),
 ('38562', '29039', -0.4472135954999579),
 ('20866', '6195', -0.7071067811865475),
 ('49153', '39599', 0.447213595499958),
 ('49153', '6261', -0.31622776601683794),
 ('49153', '27421', 0.7071067811865475),
 ('49153', '48487', 0.7071067811865475),
 ('49154', '8361', 0.31622776601683794),
 ('42171', '44721', -0.5999999999999999),
 ('12201', '33646', 0.9999999999999998),
 ('12201', '40108', -0.9999999999999998),
 ('12201', '21577', 0.8320502943378437),
 ('26835', '33451', -0.7999999999999998),
 ('26832', '35494', 0.7071067811865475),
 ('26832',

In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
spark = SparkSession(sc)

NameError: name 'sc' is not defined

In [ ]:
pearson_rdd = sc.parallelize(pearson_between_users)

In [ ]:

pearson_rdd = sc.parallelize(pearson_between_users)  


# convert to code and Use below to dump user-user pearson to file

In [ ]:
pearson_rdd

In [ ]:
def p(x):
    print(x)
    
pearson_rdd.foreach(p)

In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
%matplotlib inline
import matplotlib.pyplot as plt
#unweighted graph creation

In [ ]:
pearson_rdd=pearson_rdd.toDF()

In [ ]:
pearson_df=pearson_rdd.toPandas()

In [ ]:
pearson_df.head(10)

In [ ]:
pearson_df.count()

In [ ]:
pearson_df["_3"].describe()

In [ ]:
#pearson_df.to_csv("USER_USER_30_100.csv")

In [ ]:
pearson_df["_3"].value_counts().plot.pie()
plt.gca().set_aspect("equal")


In [ ]:
boxplot = pearson_df.boxplot(column=["_3"])

In [ ]:
data = pearson_df[(pearson_df._3 > 0) ]

In [ ]:
data.count()

In [ ]:
G = nx.from_pandas_edgelist(data, "_1", "_2", ["_3"])

In [ ]:
nx.draw(G)
plt.show()

In [ ]:
nx.classes.function.density(G)

In [ ]:
comp = nx.algorithms.community.centrality.girvan_newman(G)

In [ ]:
cnt=0
d={}

for c in next(comp):
    #print(c)
    d[cnt]=len(c)
    #print(len(c))
    #print("\n")
    cnt+=1
print("number of communities"+str(cnt))    
print(d)
    

In [127]:
comp

<generator object girvan_newman at 0xa61b40eb0>

In [ ]:

>>> G = nx.path_graph(10)
>>> edges = G.edges()
>>> nx.set_edge_attributes(G, {(u, v): v for u, v in edges}, 'weight')
>>> def heaviest(G):
...     u, v, w = max(G.edges(data='weight'), key=itemgetter(2))
...     return (u, v)
...
>>> comp = girvan_newman(G, most_valuable_edge=heaviest)

In [ ]:
#weighted community creation
from operator import itemgetter
G = nx.from_pandas_edgelist(data, "_1", "_2", ["_3"])
nx.draw(G)
plt.show()
nx.classes.function.density(G)
def heaviest(G):
    u, v, w = min(G.edges(data='_3'), key=itemgetter(2))
    return (u, v)

comp = girvan_newman(G, most_valuable_edge=heaviest)